In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

# -----------------------------
# 1. داده مصنوعی
# -----------------------------
np.random.seed(42)
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

X_flat = X.flatten()
y_flat = y.flatten()

# -----------------------------
# 2. تابع هزینه
# -----------------------------
def compute_cost(y, y_pred):
    m = len(y)
    return (1/(2*m)) * np.sum((y_pred - y)**2)

# -----------------------------
# 3. گرادیان نزولی
# -----------------------------
def gradient_descent(X, y, a, b, lr, n_iters):
    m = len(y)
    cost_history = []
    params_history = []

    for _ in range(n_iters):
        y_pred = a * X + b
        da = -(2/m) * np.sum(X * (y - y_pred))
        db = -(2/m) * np.sum(y - y_pred)
        a -= lr * da
        b -= lr * db
        cost_history.append(compute_cost(y, y_pred))
        params_history.append((a, b))
    return a, b, cost_history, params_history

# -----------------------------
# 4. اجرای گرادیان نزولی
# -----------------------------
a_init, b_init = 0, 0
learning_rate = 0.05
n_iters = 50

a_final, b_final, cost_history, params_history = gradient_descent(
    X_flat, y_flat, a_init, b_init, learning_rate, n_iters
)

x_line = np.linspace(0, 2, 100)

# -----------------------------
# 5. آماده‌سازی سطح تابع هزینه
# -----------------------------
a_vals = np.linspace(0, 6, 50)
b_vals = np.linspace(0, 6, 50)
A, B = np.meshgrid(a_vals, b_vals)
Z = np.zeros_like(A)

for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        y_pred_grid = A[i,j]*X_flat + B[i,j]
        Z[i,j] = compute_cost(y_flat, y_pred_grid)

# -----------------------------
# 6. آماده‌سازی نمودارها
# -----------------------------
fig = plt.figure(figsize=(18,6))

# subplot 1: خط روی داده‌ها
ax1 = fig.add_subplot(1,3,1)
scatter = ax1.scatter(X_flat, y_flat, c='blue', label='Data')
line, = ax1.plot([], [], color='red', linewidth=2, label='Fitted Line')
ax1.set_xlim(0, 2)
ax1.set_ylim(min(y_flat)-1, max(y_flat)+1)
ax1.set_title("Line Fit")
ax1.legend()

# subplot 2: نمودار هزینه
ax2 = fig.add_subplot(1,3,2)
ax2.set_xlim(0, n_iters)
ax2.set_ylim(0, max(cost_history)+1)
ax2.set_title("Cost Function")
ax2.set_xlabel("Iteration")
ax2.set_ylabel("Cost (MSE)")
cost_line, = ax2.plot([], [], color='purple', linewidth=2)

# subplot 3: 3D سطح تابع هزینه
ax3 = fig.add_subplot(1,3,3, projection='3d')
ax3.plot_surface(A, B, Z, cmap='viridis', alpha=0.6)
point, = ax3.plot([], [], [], 'ro', markersize=6)
line_path, = ax3.plot([], [], [], 'r--', alpha=0.3)
ax3.set_xlabel('a (slope)')
ax3.set_ylabel('b (intercept)')
ax3.set_zlabel('Cost')
ax3.set_title('3D Cost Surface')

# -----------------------------
# 7. init و update
# -----------------------------
def init():
    line.set_data([], [])
    cost_line.set_data([], [])
    scatter.set_color(['blue']*len(X_flat))
    point.set_data([], [])
    point.set_3d_properties([])
    line_path.set_data([], [])
    line_path.set_3d_properties([])
    return line, cost_line, scatter, point, line_path

def update(frame):
    a, b = params_history[frame]
    y_pred_points = a * X_flat + b

    # subplot 1: خط روی داده‌ها و رنگ نقاط براساس خطا
    line.set_data(x_line, a*x_line+b)
    errors = np.abs(y_flat - y_pred_points)
    scatter.set_color(plt.cm.Reds(errors / max(errors)))

    # subplot 2: نمودار هزینه
    cost_line.set_data(np.arange(frame+1), np.array(cost_history[:frame+1]))

    # subplot 3: نقطه روی سطح و مسیر
    cost = compute_cost(y_flat, y_pred_points)
    point.set_data([a], [b])
    point.set_3d_properties([cost])

    path_a = [p[0] for p in params_history[:frame+1]]
    path_b = [p[1] for p in params_history[:frame+1]]
    path_cost = [compute_cost(y_flat, a_*X_flat + b_) for a_, b_ in params_history[:frame+1]]
    line_path.set_data(path_a, path_b)
    line_path.set_3d_properties(path_cost)

    return line, cost_line, scatter, point, line_path

# -----------------------------
# 8. اجرای انیمیشن
# -----------------------------
anim = FuncAnimation(fig, update, frames=n_iters, init_func=init, blit=False, interval=200)
plt.close()
HTML(anim.to_jshtml())


Output hidden; open in https://colab.research.google.com to view.